In [3]:
!pip install transformers


In [40]:
import numpy as np
import pandas as pd
from ekphrasis.classes.preprocessor import TextPreProcessor
from ekphrasis.dicts.emoticons import emoticons
from nltk.stem import WordNetLemmatizer
import nltk
from mittens import GloVe
from sklearn.feature_extraction.text import CountVectorizer
from scipy.spatial.distance import cosine
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score
import tensorflow as tf
tf.compat.v1.enable_eager_execution()
tf.data.experimental.enable_debug_mode()
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Bidirectional, LSTM, Dense, Dropout
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [4]:
!pip3 install torch torchvision

In [10]:
import torch
x = torch.rand(5, 3)
print(x)

tensor([[0.5134, 0.6471, 0.5653],
        [0.6019, 0.9698, 0.6065],
        [0.0940, 0.2875, 0.5034],
        [0.7584, 0.5929, 0.7469],
        [0.3057, 0.7307, 0.5586]])


# BERT Model

AttributeError: 'Series' object has no attribute 'type'

In [55]:
df=pd.read_csv("HateSpeechDataset.csv")
data = df.iloc[:1000]
data["Label"].astype(int)
# Extract the text and labels from the DataFrame
texts = data["Content"].tolist()  # Convert the 'Content' column to a list
true_labels = data["Label"].tolist()

In [57]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, pipeline
from sklearn.metrics import accuracy_score, precision_score, recall_score

# Load HateBERT or any similar fine-tuned BERT model
bert_model_name = "Hate-speech-CNERG/bert-base-uncased-hatexplain"
tokenizer = AutoTokenizer.from_pretrained(bert_model_name)
model = AutoModelForSequenceClassification.from_pretrained(bert_model_name)

# Create a pipeline for text classification
bert_pipeline = pipeline("text-classification", model=model, tokenizer=tokenizer)

'''
# Example text input
text = [
    "go to hell you stupid person",
    "I love you"
]
'''

# Perform hate speech detection
predictions = bert_pipeline(texts)
print("BERT Results:")
'''
for t, r in zip(text, bert_results):
    print(f"Text: {t}\nPrediction: {r}\n")
'''
predicted_labels = [1 if pred["label"] == "LABEL_1" else 0 for pred in predictions]

# Compute metrics
accuracy = accuracy_score(true_labels, predicted_labels)
print(f"Accuracy: {accuracy:.2f}")
precision = precision_score(true_labels, predicted_labels)
print(f"Precision: {precision:.2f}")
recall = recall_score(true_labels, predicted_labels)
print(f"Recall: {recall:.2f}")


Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


BERT Results:
Accuracy: 0.00


ValueError: Mix of label input types (string and number)

In [51]:
print(data["Label"].unique())

['1' '0']


# RoBERTa

In [11]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, pipeline

# Alternative RoBERTa model for hate speech detection
roberta_model_name = "cardiffnlp/twitter-roberta-base-hate"
tokenizer = AutoTokenizer.from_pretrained(roberta_model_name)
model = AutoModelForSequenceClassification.from_pretrained(roberta_model_name)

# Create a pipeline for text classification
roberta_pipeline = pipeline("text-classification", model=model, tokenizer=tokenizer)

# Example text input
text = [
    "Everyone deserves to be treated with respect and kindness.",
    "you deserve to die."
]

# Perform hate speech detection
roberta_results = roberta_pipeline(text)
print("\nRoBERTa Results:")
for t, r in zip(text, roberta_results):
    print(f"Text: {t}\nPrediction: {r}\n")


Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.



RoBERTa Results:
Text: Everyone deserves to be treated with respect and kindness.
Prediction: {'label': 'non-hate', 'score': 0.9583208560943604}

Text: you deserve to die.
Prediction: {'label': 'non-hate', 'score': 0.8302949666976929}



# GPT4All

In [ ]:
from gpt4all import GPT4All
model = GPT4All("Phi-3-mini-4k-instruct.Q4_0.gguf") # downloads / loads a 4.66GB LLM

In [2]:
with model.chat_session():
    print(model.generate("classify this sentence as hate speech or normal: the fuck is this vagina on about fuck mike ashley the prick nfc. Only reply with 'hateful' or 'non hateful'.", max_tokens=1024))

 hateful
response=The provided statement contains explicit language and derogatory terms, which classifies it under hate speech due to its disrespectful nature towards an individual mentioned within the sentence. Therefore:


hateful


It is important to note that while this classification focuses on the presence of offensive content, legal definitions may vary by jurisdiction and context for what constitutes hate speech specifically.


In [6]:
from gpt4all import GPT4All
import pandas as pd

# Load your model
model = GPT4All("Phi-3-mini-4k-instruct.Q4_0.gguf")

# Load your dataset
# Assuming the dataset is a CSV file with a column 'Content'
df=pd.read_csv("HateSpeechDataset.csv")
df = df.iloc[:1000]

# Create a new column to store predictions
df['classification'] = None

# Process each sentence
with model.chat_session():
    for idx, row in df.iterrows():
        sentence = row['Content']
        # Generate prediction
        prompt = f"classify this sentence as hate speech or normal: {sentence}. Only reply with 'hateful' or 'non hateful'."
        prediction = model.generate(prompt, max_tokens=10).strip()
        df.at[idx, 'classification'] = prediction

# Save the results to a new CSV file
#df.to_csv('classified_sentences.csv', index=False)


Llama: context full, swapping: n_past = 2046, n_keep = 1, n_discard = 1536
Llama: context full, swapping: n_past = 2048, n_keep = 1, n_discard = 1536
Llama: context full, swapping: n_past = 2047, n_keep = 1, n_discard = 1536
Llama: context full, swapping: n_past = 2048, n_keep = 1, n_discard = 1536
Llama: context full, swapping: n_past = 2044, n_keep = 1, n_discard = 1536
LLaMA ERROR: Failed to process prompt
LLaMA ERROR: Failed to process prompt
LLaMA ERROR: Failed to process prompt
LLaMA ERROR: Failed to process prompt
LLaMA ERROR: Failed to process prompt
LLaMA ERROR: Failed to process prompt
LLaMA ERROR: Failed to process prompt
LLaMA ERROR: Failed to process prompt
LLaMA ERROR: Failed to process prompt
LLaMA ERROR: Failed to process prompt
LLaMA ERROR: Failed to process prompt
LLaMA ERROR: Failed to process prompt
LLaMA ERROR: Failed to process prompt
LLaMA ERROR: Failed to process prompt
LLaMA ERROR: Failed to process prompt
LLaMA ERROR: Failed to process prompt
LLaMA ERROR: Fail

In [5]:
prediction

''

In [ ]:
# Calculate metrics
from sklearn.metrics import accuracy_score, precision_score, recall_score
true_labels = df['Label']
predicted_labels = df['classification']

accuracy = accuracy_score(true_labels, predicted_labels)
precision = precision_score(true_labels, predicted_labels, pos_label='1')
recall = recall_score(true_labels, predicted_labels, pos_label='1')

print(f"Accuracy: {accuracy:.2f}")
print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")

Trials

In [58]:
# Convert labels from strings to integers
data["Label"] = data["Label"].astype(int)

# Verify the conversion
print(data["Label"].unique())  # Should now output [1, 0]

# Extract texts and labels
texts = data["Content"].tolist()
true_labels = data["Label"].tolist()

# Proceed with the rest of your code
from transformers import AutoTokenizer, AutoModelForSequenceClassification, pipeline
from sklearn.metrics import accuracy_score, precision_score, recall_score

# Load the HateBERT model
bert_model_name = "Hate-speech-CNERG/bert-base-uncased-hatexplain"
tokenizer = AutoTokenizer.from_pretrained(bert_model_name)
model = AutoModelForSequenceClassification.from_pretrained(bert_model_name)

# Create a pipeline for text classification
bert_pipeline = pipeline("text-classification", model=model, tokenizer=tokenizer)

# Perform predictions on the texts
predictions = bert_pipeline(texts)

# Convert predictions to binary labels
predicted_labels = [1 if pred["label"] == "LABEL_1" else 0 for pred in predictions]

# Compute metrics
accuracy = accuracy_score(true_labels, predicted_labels)
precision = precision_score(true_labels, predicted_labels)
recall = recall_score(true_labels, predicted_labels)

# Print results
print(f"Accuracy: {accuracy:.2f}")
print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")


/var/folders/y8/p7vw8wn924qchdnzwptmkrp40000gn/T/ipykernel_99794/2772178208.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["Label"] = data["Label"].astype(int)


[1 0]


Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


Accuracy: 0.41
Precision: 0.00
Recall: 0.00


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
